In [2]:
import glob
import os

In [3]:
xlxs_path = '/Users/soiji/Library/CloudStorage/GoogleDrive-sooh0601@gmail.com/내 드라이브/Data Study/Data Preprocessing Practice/Data'
xlxs_list = os.listdir('Data')

In [4]:
import pandas as pd
import numpy as np

### Problem 1
Re-arrange data into the correct four columns
Columns would be = Segment, Ship Mode, OrderID, Sales
Delete Grand Totals and Sub Totals Data

In [5]:
data = pd.read_excel(xlxs_path + '/' + str(xlxs_list[0]), engine='openpyxl')

전략을 구성해보자.
테이블이 아예 피봇된 형태인거 아님?
index = OrderID
colum = Segment, Shipmode
Value = Cost

In [7]:
# 필요없는 총합 데이터부터 제거합시다
drop_col = [c for c in data.columns if 'Total' in c]
data.drop(drop_col, axis=1, inplace=True)
data.drop(824, axis=0, inplace=True)

In [50]:
# Segment에 따라 Table 분류합시다
df_consumer = data.iloc[:, [0, 1, 2, 3, 4]].copy()
df_corporate = data.iloc[:, [0, 5, 6, 7, 8]].copy()
df_homeoffice = data.iloc[:, [0, 9, 10, 11, 12]].copy()

# LOOP 변수 정의
df_list = [df_consumer, df_corporate, df_homeoffice]
df_name = ['df_consumer', 'df_corporate', 'df_homeoffice']

final_frame = pd.DataFrame([])

# Table Name 지정
for i, d in enumerate(df_list):
    d.name = df_name[i]

# Table 편집
for df in df_list:
    name = df.name[3:]
    new_col = df.iloc[0].tolist()[1:]
    new_index = pd.Series(df.iloc[2:, 0].tolist(), name='Order ID')
    value = df.iloc[2:,1:].to_numpy()
    df = pd.DataFrame(data = value, index = new_index, columns = new_col)

    df['Ship Mode'] = np.where(df['First Class'].isna() == False, 'First Class', np.where(df['Same Day'].isna() == False, 'Same Day', 
    np.where(df['Second Class'].isna() == False, 'Second Class', np.where(df['Standard Class'].isna() == False, 'Standard Class', None))))
    df['Sales'] = np.where(df['First Class'].isna() == False, df['First Class'], np.where(df['Same Day'].isna() == False, df['Same Day'], 
    np.where(df['Second Class'].isna() == False, df['Second Class'], np.where(df['Standard Class'].isna() == False, df['Standard Class'], None)))) 
    df['Segment'] = name

    df.drop(columns = df.columns[0:4], inplace=True)
    df = df[df['Ship Mode'].isna() == False]
    final_frame = pd.concat([final_frame, df])

In [60]:
final_frame = final_frame[['Segment', 'Ship Mode', 'Sales']]

In [61]:
final_frame

,Segment,Ship Mode,Sales
Order ID,,,
CA-2011-100706,consumer,Second Class,129.44
CA-2011-100895,consumer,Standard Class,605.47
CA-2011-101266,consumer,Second Class,13.36
CA-2011-103366,consumer,First Class,149.95
CA-2011-104283,consumer,Standard Class,616.14
...,...,...,...
US-2014-132675,homeoffice,Standard Class,148.16
US-2014-148768,homeoffice,Second Class,71.976
US-2014-156083,homeoffice,Standard Class,9.664


In [72]:
#multi level indexing

col_1 = ['Consumer', 'Corporate','Home Office']
col_2 = ['First Class', 'Same Day', 'Second Class', 'Standard Class']
cols = pd.MultiIndex.from_product([col_1, col_2], names=['Segment', 'Ship Mode'])
index = pd.Series(data.iloc[2:, 0].tolist(), name='Order ID')

value = data.iloc[2:, 1:].copy().to_numpy()

df = pd.DataFrame(data=value, columns=cols, index=index)


In [81]:
pd.DataFrame(df.unstack().dropna()).pivot_table()

0
Segment     Ship Mode      Order ID               
Consumer    First Class    CA-2011-103366   149.95
                           CA-2011-109043    243.6
                           CA-2011-113166    9.568
                           CA-2011-124023     8.96
                           CA-2011-130155     34.2
...                                            ...
Home Office Standard Class US-2014-129224    4.608
                           US-2014-132031  513.496
                           US-2014-132297   598.31
                           US-2014-132675   148.16
                           US-2014-156083    9.664

[822 rows x 1 columns]